In [ ]:
# import the relevant .csv files
import pandas as pd
tdf_riders = pd.read_csv('TDF_Riders_Info.csv')

In [ ]:
all_tdf_participated_in = tdf_riders['Tour de Frances participated in']
tdf_years = []

for tdfs_participated_in in all_tdf_participated_in:
    tdfs_participated_in_list = tdfs_participated_in.split(", ")
    for tdf_year in tdfs_participated_in_list:
        if int(tdf_year) not in tdf_years:
            tdf_years.append(int(tdf_year))

tdf_years.sort()

# all the Tour de France editions that the available riders have raced in (need to get route information for all stages in each of these Tours)
print(f"Tour de Frances: {tdf_years}")
print(f"Number of Tour de Frances: {len(tdf_years)}")

In [ ]:
# add the total finish times of each Tour de France edition for each rider in the TDF_Riders dataframe
tdf_riders_history = pd.read_csv('TDF_Riders_History.csv')

for tdf_year in tdf_years:
    for rider_index in range(len(tdf_riders['Rider'])):
        rider = tdf_riders.iloc[rider_index]['Rider']
        tdfs_participated_in_raw = tdf_riders.iloc[rider_index]['Tour de Frances participated in'].split(", ")
        tdfs_participated_in = []
    
        for tdf_edition_raw in tdfs_participated_in_raw:
            tdfs_participated_in.append(int(tdf_edition_raw))

        if tdf_year not in tdfs_participated_in:
            continue
        else:
            tdf_edition_finishes_dataframe = tdf_riders_history.loc[tdf_riders_history['Year'] == tdf_year]
            finish_time = tdf_edition_finishes_dataframe.loc[tdf_edition_finishes_dataframe['Rider'] == rider.upper()]['TotalSeconds']
            finish_time = finish_time.to_string().split(" ")[-1]
            tdf_riders.at[rider_index, f'{tdf_year} TDF Total Finish Time (s)'] = finish_time

tdf_riders.to_csv('TDF_Riders_Timings.csv', sep=',', encoding='utf-8', index=False)